# 코드 정리

In [8]:
import pandas as pd
import holidays
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import mean_squared_error
import xgboost as xgb

In [139]:
# !pip3 install xgboost
# !pip3 uninstall lightgbm -y
# !pip3 uninstall pytimekr -y
# !pip3 install holidays

Found existing installation: pytimekr 0.1.0
Uninstalling pytimekr-0.1.0:
  Successfully uninstalled pytimekr-0.1.0


### data load

In [68]:
df1 = pd.read_csv("../dataset/international_trade.csv")
df2 = pd.read_csv("../dataset/sample_submission.csv")
df3 = pd.read_csv("../dataset/test.csv")
df4 = pd.read_csv("../dataset/train.csv")

## preprocessing

### analyze
- train 데이터 : 2019년 01월 01일부터 2023년 03월 03일까지의 유통된 품목의 가격 데이터
- item: 품목 코드
    - TG : 감귤
    - BC : 브로콜리
    - RD : 무
    - CR : 당근
    - CB : 양배추
- corporation : 유통 법인 코드
    - 법인 A부터 F 존재
- location : 지역 코드
    - J : 제주도 제주시
    - S : 제주도 서귀포시
- supply(kg) : 유통된 물량, kg 단위
- price(원/kg) : 유통된 품목들의 kg 마다의 가격, 원 단위

In [221]:
df4.head()

,ID,timestamp,item,corporation,location,supply(kg),price(원/kg)
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0


In [222]:
for i in ["item","corporation","location"]:
    print(df4[i].value_counts())
    print()

item
TG    15230
BC    13707
RD    12184
CR    10661
CB     7615
Name: count, dtype: int64

corporation
A    13707
E    13707
D    12184
C     9138
B     7615
F     3046
Name: count, dtype: int64

location
J    35029
S    24368
Name: count, dtype: int64



### 시계열로 감귤(TG)의 특정 회사(A)의 2019년 1월 한달을 그려봤다
- 가격이 0인 날이 꽤 있다. -> 살펴 봐야겠다
- 1월에 점점 가격이 올라간다

In [223]:
import matplotlib.pyplot as plt

In [224]:
cond1 = df['item'] =='BC'
cond2 = df['corporation'] == 'A'
cond3 = df['month'] == 1
cond4 = df['year'] == 2019
test = df[cond1 & cond2 & cond3 & cond4]

In [225]:
plt.figure(figsize=(12, 6))  # 그래프 크기 설정

plt.plot(test['timestamp'], test['price(원/kg)'],'o', linestyle='-', color='b', label='Price')  # 그래프 그리기

plt.title('Price Over Time')  # 그래프 제목 설정
plt.xlabel('Timestamp')  # x축 레이블 설정
plt.ylabel('Price(원/kg)')  # y축 레이블 설정

plt.grid(True)  # 그리드 표시
plt.legend()  # 범례 표시

plt.xticks(rotation=45)  # x축 레이블 회전

plt.show()  # 그래프 표시

KeyError: 'timestamp'

<Figure size 1200x600 with 0 Axes>

### 가격이 0인 날짜들을 살펴보았다
- 휴일 (공휴일, 일요일)이면 무조건 가격이 0이다
- 평일에도 안판 법인이 있을 수 있다

In [137]:
cond1 = df['price(원/kg)'] == 0.0
# cond2 = df["timestamp"] == "2019-01-01"
df[cond1]

,ID,timestamp,item,corporation,location,supply(kg),price(원/kg),year,month,day,holiday
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,2019,1,1,1
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0,2019,1,2,0
5,TG_A_J_20190106,2019-01-06,TG,A,J,0.0,0.0,2019,1,6,0
12,TG_A_J_20190113,2019-01-13,TG,A,J,0.0,0.0,2019,1,13,0
19,TG_A_J_20190120,2019-01-20,TG,A,J,0.0,0.0,2019,1,20,0
...,...,...,...,...,...,...,...,...,...,...,...
59363,RD_F_J_20230129,2023-01-29,RD,F,J,0.0,0.0,2023,1,29,0
59370,RD_F_J_20230205,2023-02-05,RD,F,J,0.0,0.0,2023,2,5,0
59377,RD_F_J_20230212,2023-02-12,RD,F,J,0.0,0.0,2023,2,12,0
59384,RD_F_J_20230219,2023-02-19,RD,F,J,0.0,0.0,2023,2,19,0


### 수치화

In [191]:
df4 = df4.drop('ID',axis = 1)

In [192]:
def categoryChange(df):
    """ 카테고리화 """
    ordinal_encoder = OrdinalEncoder()
    ordinal = ordinal_encoder.fit_transform(df)
    return ordinal

In [193]:
for name in ['item','corporation','location']:
    df4.loc[:,name] = categoryChange(df4[[name]])
    df4[name] = pd.to_numeric(df4[name])

### 날짜 분리

In [204]:
def dateChange(df):
    df.loc[:,'year'] = df['timestamp'].dt.year
    df.loc[:,'month'] = df['timestamp'].dt.month
    df.loc[:,'day'] = df['timestamp'].dt.day
    return df 

In [43]:
df4 = dateChange(df4)

### 휴일 처리

In [ ]:
import holidays
# 한국 휴일 객체 생성 
kr_holidays = holidays.KR()

df.loc[:,'holiday'] = df['timestamp'].apply(lambda a: 1 if a in kr_holidays else 0)
df.loc[:,'sunday'] = df['timestamp'].apply(lambda a: 1 if a.weekday()==6 else 0)
df.loc[:,"holisunDay"] = pd.concat([df["holiday"], df["sunday"]],axis = 0,ignore_index=True)

In [185]:
df.drop(['holiday','sunday','timestamp'],axis = 1,inplace = True)

### model training

In [195]:
X = df4.drop(['price(원/kg)','supply(kg)'],axis = 1)
Y = df4['price(원/kg)']

In [196]:
x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size=0.2, random_state=42,stratify= df4['item'])

In [197]:
model = RandomForestRegressor()

In [198]:
model = xgb.XGBRegressor()

In [199]:
model.fit(x_train,y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [200]:
pred = model.predict(x_test)

In [201]:
"""
LinearRegressor -> 1904.0070927607574
RandomForestRegressor -> 1580.9442694709762
xgbregressor -> 1580.9064095937372

RandomForestRegressor -> 1192.357782768199 (날짜 전처리)
xgbregressor -> 1148.0481545075852 (날짜 전처리)
xgbregressor -> 1145.1369208445203 (날짜 전처리2)
"""
forest_mse = mean_squared_error(y_test, pred)
forest_rmse = np.sqrt(forest_mse)
forest_rmse

1145.1369208445203

### Submission

In [142]:
import holidays
def holisunDay(df):
    """ 한국 휴일, 일요일 피쳐 생성 """
    kr_holidays = holidays.KR()
    df.loc[:,'holiday'] = df['timestamp'].apply(lambda a: 1 if a in kr_holidays else 0)
    df.loc[:,'sunday'] = df['timestamp'].apply(lambda a: 1 if a.weekday()==6 else 0)
    df.loc[:,"holisunDay"] = pd.concat([df["holiday"], df["sunday"]],axis = 0,ignore_index=True)
    df.drop(['holiday','sunday','timestamp'],axis = 1,inplace = True)
    return df

In [143]:
def dateChange(df):
    """ 시간 분리 """  
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df.loc[:,'year'] = df['timestamp'].dt.year
    df.loc[:,'month'] = df['timestamp'].dt.month
    df.loc[:,'day'] = df['timestamp'].dt.day
    return df 

In [144]:
def categoryChange(df):
    """ 카테고리화 """
    ordinal_encoder = OrdinalEncoder()
    ordinal = ordinal_encoder.fit_transform(df)
    return ordinal

In [150]:
def changeNumeric(s):
    v = pd.DataFrame(s, columns = cat_attribs+other)
    df = v[other + cat_attribs]
    for name in ['item','corporation','location']:
        df[name] = pd.to_numeric(df[name])
    return df

In [146]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer

In [155]:
df = df3.copy()

In [156]:

cat_attribs = ['item','corporation','location']
other = ['ID','timestamp']

# 수치화 파이프라인
col_pipeline = ColumnTransformer([
    ("ordinal", Pipeline([("encoder", OrdinalEncoder())]), cat_attribs),
],remainder="passthrough")

#날짜 및 휴일 피쳐 생성 파이프라인
feature_pipeline = Pipeline([
    ('dateChange', FunctionTransformer(func=dateChange, validate=False)),
    ('holisunDay', FunctionTransformer(func=holisunDay, validate=False))
])

s = col_pipeline.fit_transform(df)
df = changeNumeric(s)
df = feature_pipeline.transform(df)

### Modeling

In [212]:
X = df3.drop(['ID'],axis = 1)

In [213]:
real = model.predict(X)

In [214]:
df2

,ID,answer
0,TG_A_J_20230304,0
1,TG_A_J_20230305,0
2,TG_A_J_20230306,0
3,TG_A_J_20230307,0
4,TG_A_J_20230308,0
...,...,...
1087,RD_F_J_20230327,0
1088,RD_F_J_20230328,0
1089,RD_F_J_20230329,0
1090,RD_F_J_20230330,0


In [215]:
df2.loc[:,'answer'] = real

In [216]:
df2['answer']

0       2947.173584
1       1696.663208
2       2382.136719
3       2962.096924
4       3452.773438
           ...     
1087     439.648499
1088     917.850708
1089      87.716927
1090    1146.545044
1091    1189.047241
Name: answer, Length: 1092, dtype: float32

In [217]:
df2.to_csv("answer.csv",index = False)